In [9]:
import math
import pandas as pd
import os

import glob
from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq
import time


import psycopg2 as pg2
from sqlalchemy import create_engine

from datetime import datetime
from datetime import timedelta

import os
import time

import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


In [4]:
# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


### GCS 데이터 등록

In [10]:

# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'

# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력


### csv, parquet

- [parquet 공식문서](https://arrow.apache.org/docs/python/parquet.html)
- [Python parquet 타입으로 읽기, 쓰기](https://owening2.tistory.com/19)
- [parquet? / csv, parquet 파일 불러오기 테스트](https://velog.io/@freetix/parquet-csv-parquet-%ED%8C%8C%EC%9D%BC-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0-%ED%85%8C%EC%8A%A4%ED%8A%B8
)


In [2]:
os.getcwd()

'/home/shjj08choi4/finance_mlops/jupyter'

In [5]:
kor_stock_ohlcv = pd.read_csv('data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.csv')

/tmp/ipykernel_883218/2625704097.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  kor_stock_ohlcv = pd.read_csv('data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.csv')


In [13]:
kor_stock_ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3366395 entries, 0 to 3366394
Data columns (total 8 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   date                     object 
 1   open                     int64  
 2   high                     int64  
 3   low                      int64  
 4   close                    int64  
 5   volume                   int64  
 6   price_change_percentage  float64
 7   ticker                   object 
dtypes: float64(1), int64(5), object(2)
memory usage: 205.5+ MB


In [13]:
kor_stock_ohlcv.head()

,date,open,high,low,close,volume,price_change_percentage,ticker
0,2018-01-02,6980,6990,6850,6970,60294,0.000000,095570
1,2018-01-03,6970,7000,6910,6920,45848,-0.717360,095570
2,2018-01-04,6990,7100,6920,7070,53176,2.167630,095570
3,2018-01-05,7080,7300,7040,7250,58573,2.545969,095570
4,2018-01-08,7330,7330,7100,7130,47104,-1.655172,095570


In [14]:
kor_stock_ohlcv['ticker'] = kor_stock_ohlcv['ticker'].astype('str')

#### parquet로 저장하기

In [15]:

table_from_pandas = pa.Table.from_pandas(kor_stock_ohlcv,preserve_index = False)
pq.write_table(table_from_pandas, 'data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.parquet')

/home/shjj08choi4/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


#### parquet 읽어오기

In [21]:
t = pq.read_table('data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.parquet')

In [22]:
t

pyarrow.Table
date: string
open: int64
high: int64
low: int64
close: int64
volume: int64
price_change_percentage: double
ticker: string
----
date: [["2018-01-02","2018-01-03","2018-01-04","2018-01-05","2018-01-08",...,"2023-05-23","2023-05-24","2023-05-25","2023-05-26","2023-05-30"],["2023-05-31","2023-06-01","2023-06-02","2023-06-05","2023-06-07",...,"2018-04-23","2018-04-24","2018-04-25","2018-04-26","2018-04-27"],...,["2022-01-13","2022-01-14","2022-01-17","2022-01-18","2022-01-19",...,"2019-02-07","2019-02-08","2019-02-11","2019-02-12","2019-02-13"],["2019-02-14","2019-02-15","2019-02-18","2019-02-19","2019-02-20",...,"2023-11-30","2023-12-01","2023-12-04","2023-12-05","2023-12-06"]]
open: [[6980,6970,6990,7080,7330,...,16000,15740,15610,15600,15670],[16100,15700,16080,15980,16560,...,15500,14800,14600,14750,14850],...,[5030,4890,4980,4935,4730,...,2735,2820,2820,2825,2860],[2880,2945,2920,2910,2995,...,6220,6330,6450,6480,6280]]
high: [[6990,7000,7100,7300,7330,...,16040,15830,157

In [17]:
t.to_pandas()

,date,open,high,low,close,volume,price_change_percentage,ticker
0,2018-01-02,6980,6990,6850,6970,60294,0.000000,095570
1,2018-01-03,6970,7000,6910,6920,45848,-0.717360,095570
2,2018-01-04,6990,7100,6920,7070,53176,2.167630,095570
3,2018-01-05,7080,7300,7040,7250,58573,2.545969,095570
4,2018-01-08,7330,7330,7100,7130,47104,-1.655172,095570
...,...,...,...,...,...,...,...,...
3366390,2023-11-30,6220,6330,6210,6330,12703,2.261712,238490
3366391,2023-12-01,6330,6490,6290,6410,31437,1.263823,238490
3366392,2023-12-04,6450,6500,6380,6430,19005,0.312012,238490
3366393,2023-12-05,6480,6480,6250,6360,21136,-1.088647,238490


In [18]:
# Google Storage 적재
source_file_name = f'data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.parquet'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)